In [1]:
import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import artool
from artool import toy, analyze
from artool.toy.toy_simu import get_pnl_2side

# remove limits on number of rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

import matplotlib

matplotlib.use("Agg")  # use to improve performance
import matplotlib.pyplot as plt
import seaborn as sns


## Data

In [2]:
data_dir = Path("/home/yangzhe/data/toy_data_2")
date_start = datetime.datetime(2022, 1, 1)
date_end = datetime.datetime(2022, 9, 1)
symbols = toy.toy_data.get_symbol_list(date_start, date_end, logic="and")
print(f"number of symbols: {len(symbols)}")

df = pd.DataFrame()
for symbol in symbols:
    df_ = pd.read_feather(data_dir / f"{symbol}.feather")
    # remove head/tail 5 rows
    df_ = df_.iloc[5:-5]
    df = pd.concat([df, df_], axis=0)
df = df.reset_index(drop=True)


number of symbols: 129


In [3]:
df.head()

,symbol,funding_timestamp,funding_rate,index_price,mark_price,funding_rate_expcumsum_0.5,funding_rate_expcumsum_1,funding_rate_expcumsum_2,funding_rate_expcumsum_3,funding_rate_expcumsum_5,funding_rate_expcumsum_10,funding_rate_expcumsum_20,funding_rate_expcumsum_50,funding_rate_rol_mean_3,funding_rate_rol_std_3,funding_rate_rol_max_3,funding_rate_rol_min_3,funding_rate_rol_skew_3,funding_rate_rol_mean_5,funding_rate_rol_std_5,funding_rate_rol_max_5,funding_rate_rol_min_5,funding_rate_rol_skew_5,funding_rate_rol_kurt_5,funding_rate_rol_mean_10,funding_rate_rol_std_10,funding_rate_rol_max_10,funding_rate_rol_min_10,funding_rate_rol_skew_10,funding_rate_rol_kurt_10,funding_rate_rol_mean_20,funding_rate_rol_std_20,funding_rate_rol_max_20,funding_rate_rol_min_20,funding_rate_rol_skew_20,funding_rate_rol_kurt_20,funding_rate_rol_mean_50,funding_rate_rol_std_50,funding_rate_rol_max_50,funding_rate_rol_min_50,funding_rate_rol_skew_50,funding_rate_rol_kurt_50,funding_rate_future_1,funding_rate_future_3,funding_rate_future_5,funding_rate_future_10,index_price_expcumsum_0.5,index_price_expcumsum_1,index_price_expcumsum_2,index_price_expcumsum_3,index_price_expcumsum_5,index_price_expcumsum_10,index_price_expcumsum_20,index_price_expcumsum_50,index_price_rol_mean_3,index_price_rol_std_3,index_price_rol_max_3,index_price_rol_min_3,index_price_rol_skew_3,index_price_rol_mean_5,index_price_rol_std_5,index_price_rol_max_5,index_price_rol_min_5,index_price_rol_skew_5,index_price_rol_kurt_5,index_price_rol_mean_10,index_price_rol_std_10,index_price_rol_max_10,index_price_rol_min_10,index_price_rol_skew_10,index_price_rol_kurt_10,index_price_rol_mean_20,index_price_rol_std_20,index_price_rol_max_20,index_price_rol_min_20,index_price_rol_skew_20,index_price_rol_kurt_20,index_price_rol_mean_50,index_price_rol_std_50,index_price_rol_max_50,index_price_rol_min_50,index_price_rol_skew_50,index_price_rol_kurt_50,mark_price_expcumsum_0.5,mark_price_expcumsum_1,mark_price_expcumsum_2,mark_price_expcumsum_3,mark_price_expcumsum_5,mark_price_expcumsum_10,mark_price_expcumsum_20,mark_price_expcumsum_50,mark_price_rol_mean_3,mark_price_rol_std_3,mark_price_rol_max_3,mark_price_rol_min_3,mark_price_rol_skew_3,mark_price_rol_mean_5,mark_price_rol_std_5,mark_price_rol_max_5,mark_price_rol_min_5,mark_price_rol_skew_5,mark_price_rol_kurt_5,mark_price_rol_mean_10,mark_price_rol_std_10,mark_price_rol_max_10,mark_price_rol_min_10,mark_price_rol_skew_10,mark_price_rol_kurt_10,mark_price_rol_mean_20,mark_price_rol_std_20,mark_price_rol_max_20,mark_price_rol_min_20,mark_price_rol_skew_20,mark_price_rol_kurt_20,mark_price_rol_mean_50,mark_price_rol_std_50,mark_price_rol_max_50,mark_price_rol_min_50,mark_price_rol_skew_50,mark_price_rol_kurt_50
0,1INCHUSDT,1.641168e+15,0.0001,2.555800,2.553690,0.000016,0.000057,0.000127,0.000169,0.000210,0.000243,0.000258,0.000266,0.0001,0.0,0.0001,0.0001,NaN,0.00008,4.381799e-05,0.0001,0.000002,-2.236068,5.0,0.000045,0.000095,0.0001,-0.000131,-1.722778,2.449991,0.000045,0.000095,0.0001,-0.000131,-1.722778,2.449991,0.000045,0.000095,0.0001,-0.000131,-1.722778,2.449991,0.0001,0.0003,0.0005,0.000649,0.398767,1.471091,3.684687,5.475949,7.876804,10.671087,12.552391,13.884777,2.526944,0.032202,2.555800,2.492206,-0.794579,2.496316,0.048747,2.555800,2.436312,0.038232,-1.736673,2.478711,0.061325,2.555800,2.390683,-0.184950,-0.942066,2.478711,0.061325,2.555800,2.390683,-0.184950,-0.942066,2.478711,0.061325,2.555800,2.390683,-0.184950,-0.942066,0.398449,1.470015,3.682155,5.472192,7.871317,10.663479,12.543303,13.874619,2.525463,0.032064,2.553690,2.490600,-0.891507,2.494815,0.048740,2.553690,2.434683,0.022356,-1.787568,2.476884,0.061883,2.553690,2.387231,-0.220803,-0.906635,2.476884,0.061883,2.553690,2.387231,-0.220803,-0.906635,2.476884,0.061883,2.553690,2.387231,-0.220803,-0.906635
1,1INCHUSDT,1.641197e+15,0.0001,2.552671,2.552706,0.000016,0.000058,0.000138,0.000193,0.000254,0.000310,0.000341,0.000359,0.0001,0.0,0.0001,0.0001,NaN,0.00010,1.43803

# Check corr

In [4]:
x_feature = "funding_rate_expcumsum_5"
y_feature = "funding_rate_future_5"

corr_dict = {}
for symbol in symbols:
    x = df.loc[df["symbol"] == symbol, x_feature].values
    y = df.loc[df["symbol"] == symbol, y_feature].values
    corr_dict[symbol] = np.corrcoef(x, y)[0, 1]
# Plot correlation_distribution
fig, ax = plt.subplots()
sns.kdeplot(list(corr_dict.values()), ax=ax)
ax.set_title("x vs y correlation")
fig.savefig("corr_dist.png")

In [5]:
# find loswest 5 correlation
corr_dict_sorted = sorted(corr_dict.items(), key=lambda x: x[1])
low_cor_symbols = corr_dict_sorted[:5]
print(low_cor_symbols)

[('UNFIUSDT', 0.17267762271829357), ('STMXUSDT', 0.1739725080768929), ('ZILUSDT', 0.19360660482574152), ('CVCUSDT', 0.2110023644855364), ('OCEANUSDT', 0.22586015942273072)]


In [8]:
symb = "UNFIUSDT"
for symb, _ in low_cor_symbols:
    print(f"## symbol: {symb}")
    df_tmp = df.loc[df["symbol"] == symb]

    max_corr = 0
    max_corr_feature = ""
    y = df_tmp[y_feature].values
    for feature in df_tmp.columns:
        if feature in ["symbol", "funding_timestamp"]:
            continue
        if "future" in feature:
            continue
        x = df_tmp[feature].values
        corr = np.corrcoef(x, y)[0, 1]
        if corr > max_corr:
            max_corr = corr
            max_corr_feature = feature
    print(f"max_corr: {max_corr}, max_corr_feature: {max_corr_feature}")


## symbol: UNFIUSDT
max_corr: 0.23371708761093968, max_corr_feature: funding_rate_expcumsum_1
## symbol: STMXUSDT
max_corr: 0.228767401399035, max_corr_feature: funding_rate_expcumsum_50
## symbol: ZILUSDT
max_corr: 0.3662137445962372, max_corr_feature: funding_rate
## symbol: CVCUSDT
max_corr: 0.3092344492665794, max_corr_feature: funding_rate_rol_max_3
## symbol: OCEANUSDT
max_corr: 0.30708753899576674, max_corr_feature: funding_rate_expcumsum_0.5
